# Extrair Features

Esse arquivo usará o arquivo ExtrairFeatures da Raspberry para fazer a extração das features de datasets gravados com o ReSpeaker. A diferença é que usar a classe que existia antes não é a mesma coisa que usar exatamente o que vai ser usado pelo sistema em tempo real.

Basta colocar a pasta onde estão os WAVs. Eles têm que estar nomeados como "classe_000_0.wav" em que o último número simboliza a pasta de um KFold.

In [1]:
import sys
import numpy as np
import wave
import time
import os
from IPython.display import Audio

sys.path.append("/home/pi/Programming/IC2019/Raspberry/Python/")

from extracaoFeatures import extrairFeaturesUnicoFrame as ExtrairFeatures

/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


### Parâmetros Iniciais

In [2]:
pastaOrigemWAVs = "/home/pi/Datasets/ESC_3_ReSpeaker_1595612342.0004687/audio/"
tempoJanela     = 0.500
freqAmostragem  = 16000

### Definição da função

In [3]:
def processar(metadeInicialJanelaAtual, metadeFinalJanelaAtual, nomeArquivo):
    
    # COPIANDO OS ARRAYS NA MEMORIA PRA GARANTIR QUE AS THREADS NAO VAO ZUAR O BAGULHO
    metadeInicial = np.copy(metadeInicialJanelaAtual)
    metadeFinal   = np.copy(metadeFinalJanelaAtual)
    
    # CONVERTO A JANELA PRA FLOAT 
    # (O FROM BUFFER TEM QUE SER EM INT 16 POR CAUSA DA CODIFICACAO DO AUDIO)
    # MAS DEPOIS A GNT JOGA EM FLOAT POR CAUSA DO LIBROSA
    metadeInicialFloat = np.frombuffer(metadeInicial, dtype=np.int16).astype("float")
    metadeFinalFloat   = np.frombuffer(metadeFinal, dtype=np.int16).astype("float")
    
    # JUNTO AS METADES
    janelaCompleta = np.concatenate((metadeInicialFloat, metadeFinalFloat))
    
    # EXTRACAO DE FEATURES
    features = ExtrairFeatures(janelaCompleta, freqAmostragem)
    
    # CRIANDO A LINHA PRA COLOCARNO CSV
    linhaAtual = nomeArquivo+","
    for valor in features:
        linhaAtual += str(valor) + ","
    linhaAtual += nomeArquivo.split("_")[0]+"\n"
    
    objFile.write(linhaAtual)

### Começando o processamento

In [4]:
# ABRINDO UM CSV PRA GUARDAR AS INFORMACOES
objFile = open(pastaOrigemWAVs+str(time.time())+".csv", "w")

# ESCREVENDO UM CABECALHO
cabecalho  = "nomeArquivo,"
cabecalho += "0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,"
cabecalho += "classificacaoCorreta\n"
objFile.write(cabecalho)

# PARA CADA WAV NA PASTA
for arquivoAtual in os.listdir(pastaOrigemWAVs):
    
    # PULANDO O QUE NAO FOR WAV
    if arquivoAtual[-4:] != ".wav":
        continue
        
    print("Extraindo para o arquivo", arquivoAtual)

    # CRIANDO UM OBJETO WAV DO AUDIO ATUAL QUE QUERO LER
    objWF = wave.open(pastaOrigemWAVs+arquivoAtual, 'rb')

    # VERIFICANDO QUANTAS JANELAS EU VOU GRAVAR PRO AUDIO ATUAL
    qtdAmostrasTotal          = objWF.getnframes()
    qtdAmostrasPorJanela      = int(freqAmostragem * tempoJanela)
    qtdJanelasPrincipaisTotal = int(qtdAmostrasTotal / qtdAmostrasPorJanela)

    # CRIANDO O ARRAY DE JANELAS PARCIAIS PRA FAZER SOBREPOSICAO
    janelasParciais = []

    # COLOCANDO AS METADES DA PRIMEIRA JANELA DENTRO
    janelasParciais.append(objWF.readframes(int(qtdAmostrasPorJanela/2)))
    janelasParciais.append(objWF.readframes(int(qtdAmostrasPorJanela/2)))

    # MANDANDO PROCESSAR A PRIMEIRA JANELA
    processar(janelasParciais[0], janelasParciais[1], arquivoAtual)

    # LENDO AS PROXIMAS METADES, COMECA NO 2 PQ 0 E 1 JA FOI
    for i in range(2, qtdJanelasPrincipaisTotal*2+2):

        # LEIO A NOVA METADE
        framesAtuais = objWF.readframes(int(qtdAmostrasPorJanela/2))

        # SE A ITERACAO ATUAL FOR PAR, EU SALVO A NOVA METADE EM janelasParciais[0]
        # E MANDO JUNTAR janelasParciais[1] COM janelasParciais[0] PRA FORMAR A JANELA COMPLETA
        if i % 2 == 0:
            janelasParciais[0] = framesAtuais
            processar(janelasParciais[1], janelasParciais[0], arquivoAtual)

        # SE A ITERACAO ATUAL FOR IMPAR, EU SALVO A NOVA METADE EM janelasParciais[1]
        # E MANDO JUNTAR janelasParciais[0] COM janelasParciais[1] PRA FORMAR A JANELA COMPLETA
        else:
            janelasParciais[1] = framesAtuais
            processar(janelasParciais[0], janelasParciais[1], arquivoAtual)

# FECHANDO O CSV
objFile.close()

Extraindo para o arquivo sneezing_000_1.wav


/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Extraindo para o arquivo chainsaw_000_1.wav
Extraindo para o arquivo cryingbaby_000_1.wav
